In [571]:
import numpy as np
from datetime import datetime as dt
import pandas as pd, datetime as dt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from pandas import DataFrame

In [116]:
cd C:\Users\USER\OneDrive\Documents\GitHub\Datacracy_scala_hackathon\data

C:\Users\USER\OneDrive\Documents\GitHub\Datacracy_scala_hackathon\data


In [267]:
user = pd.read_csv('user_processed.csv',sep=',')
listening = pd.read_csv('listening_processed.csv',sep=',')
order = pd.read_csv('order_processed.csv',sep=',')
clustering = pd.read_csv('clustering_processed.csv',sep=',')


# # Processing data

In [269]:
# change column name
order = order.rename(columns = {"OrderID (PK)": "orderID", "Transaction Date" : "transaction_date","UserID (FK)" : "userID", "Bill Type": "bill_type", "Total Amount": "total_amount" })
clustering = clustering.rename(columns = {"User_ID (FK)": "userID", "Type_user": "type_user", "MainCluster_ID" : "main_cluseringID", "MainCluster_Description" : "main_clustering_description"})
listening = listening.rename(columns = {"PlaylistID (PK)": "playlistID", "Playlist Name":"playlist_name", "Category":"cat", "Sub Category" :"sub_cat", "Playlist Type" :"palylist_type","Playlist Duration (min)": "playlist_duration","Actual Duration (min)":"actual_duation","UserID (FK)":"userID", "Listening Time":"listening_time", "Listening Date" :"listening_date", "Listening Datetime":"listening_datetime"})
user = user.rename(columns = {"UserID (PK)":"userID", "VIP hay Free":"user_type", "Gender":"gender", "Year of Birth":"year_of_birth","Age Range": "age_range", "Age":"age", "Region": "region","Operation Systems":"operation_system","Registration Date":"registration_date", "Total Listerning":"total_listening", "Total Listening Time (Min)":"total_listening_time",   })

# change date time
# xu li datetime
order.transaction_date = pd.to_datetime(order['transaction_date'], format='%Y-%m-%d')
user.registration_date = pd.to_datetime(user['registration_date'], format='%Y-%m-%d')
listening.listening_datetime = pd.to_datetime(listening['listening_datetime'], format='%Y-%m-%d')

# change date by weekly
def adding_by_week(df, date_field):
    # df["by_week"] = df[[date_field]].copy()
    # df.resample('W-Mon', on=date_field).sum().reset_index().sort_values(by=date_field)
    # order1.resample('W-Mon', on='transaction_date').sum().reset_index().sort_values(by='transaction_date')
    df['by_week'] = df.apply(lambda row: row[date_field] - dt.timedelta(days=row[date_field].weekday()), axis=1)
    df = df['by_week'].groupby(df['by_week']).count()

adding_by_week(order, "transaction_date")
adding_by_week(user, "registration_date")
adding_by_week(listening, "listening_datetime")

In [262]:
## fillna for the user Table

user.operation_system = user.operation_system.fillna('none')
user["gender"] = user.gender.replace("no information","none")
user["gender"] = user.gender.replace("other","none")

## add some features related to time of the listening Table

listening['hour_listening'] = listening.listening_datetime.dt.hour
listening['dayofweek'] = listening['listening_datetime'].dt.dayofweek

listening["timepoint_of_the_day"] = listening.hour_listening.apply(lambda x : 
                                                             "late night" if x < 4 else
                                                             "early morning" if x < 7 else
                                                             "morning" if x < 11 else
                                                             "early afternoon" if x < 16 else
                                                             "afternoon" if x < 18 else
                                                             "evening"
                                                            )

listening['dayofweek'] =listening.dayofweek.apply(lambda x :  "Mon" if x == 0 else
                                                             "Tue" if x ==1 else
                                                             "Wed" if x ==2 else
                                                             "Thu" if x == 3 else
                                                             "Fri" if x == 4 else
                                                             "Sat" if x == 5 else
                                                             "Sun" 
                                                            )

listening['Workingday'] = listening["dayofweek"].apply(lambda x : 0 if x == 'Sun' or x == 'Sat' else 1)


### Tập trung phân tích nhóm paid user - high value (tăng độ engagement)
- Kèm theo 1 con số dẫn chứng của Mckegin XXX về việc giữ chân khách hàng cũ
- Process tập data lại dựa trên 3 nhóm này thôi

In [280]:
# chuyển qua dùng tập paid
user.head(3)

,userID,user_type,gender,year_of_birth,age_range,age,region,operation_system,registration_date,total_listening,total_listening_time,by_week
0,86229,free,male,NaN,No information,NaN,VN,android,2020-11-27,218,4151.3,2020-11-23
1,48053,free,male,1995.0,25 - 34,26.0,VN,ios,2020-08-24,19,278.6,2020-08-24
2,46340,free,male,1988.0,25 - 34,33.0,VN,website,2020-08-19,28,170.5,2020-08-17


In [284]:
user_money = pd.merge(user,total_amount, how='left',left_on='userID', right_on='userID')
total = pd.merge(clustering,user_money, how='left',left_on='userID', right_on='userID')
total.head()

,userID,type_user,main_cluseringID,main_clustering_description,user_type,gender,year_of_birth,age_range,age,region,operation_system,registration_date,total_listening,total_listening_time,by_week,oderID,total_amount
0,40834,free,1,Hibernating,free,no information,NaN,No information,NaN,QT,ios,2020-07-01,2,8.7,2020-06-29,NaN,NaN
1,40854,free,1,Hibernating,free,female,1999.0,18 - 24,22.0,VN,website,2020-07-01,444,20988.0,2020-06-29,NaN,NaN
2,40876,free,1,Hibernating,free,no information,NaN,No information,NaN,VN,ios,2020-07-01,3,0.8,2020-06-29,NaN,NaN
3,40883,free,1,Hibernating,free,no information,NaN,No information,NaN,VN,ios,2020-07-01,5,2.9,2020-06-29,NaN,NaN
4,40891,free,1,Hibernating,free,no information,NaN,No information,NaN,VN,ios,2020-07-01,13,71.5,2020-06-29,NaN,NaN


In [296]:
paid_user = clustering[clustering.type_user == 'paid']
list_hv_paid = ['Loyal Users','Potential Users','New Paid Users']
hv_paid = paid_user.loc[paid_user.main_clustering_description.isin(list_hv_paid)]
hv_paid

,userID,type_user,main_cluseringID,main_clustering_description
24820,40843,paid,5,New Paid Users
24821,41955,paid,5,New Paid Users
24822,44112,paid,5,New Paid Users
24823,45239,paid,5,New Paid Users
24824,45647,paid,5,New Paid Users
...,...,...,...,...
27447,98716,paid,7,Loyal Users
27448,101492,paid,7,Loyal Users
27449,102073,paid,7,Loyal Users
27450,103669,paid,7,Loyal Users


In [315]:
listening.describe()

,playlistID,playlist_duration,actual_duation,userID,Unnamed: 11
count,750098.000000,750098.000000,750098.000000,750098.000000,0.0
mean,4586.328209,565.726017,15.428639,68396.183116,NaN
std,346.636915,1339.030831,26.597340,17995.910718,NaN
min,3808.000000,1.300000,0.000000,40834.000000,NaN
25%,4387.000000,135.300000,1.300000,52016.000000,NaN
50%,4619.000000,291.300000,7.300000,66782.000000,NaN
75%,4827.000000,464.400000,19.400000,82996.000000,NaN
max,5440.000000,13971.900000,1769.600000,107219.000000,NaN


## User cases:
Dựa trên đầu sách đã đọc gần nhất => recommend đầu sách tiếp theo cho ng đó
Các cách làm
1. Input là userID (thuộc high value paid user)
2. Cho ra nhóm cluster mà user này thuộc - cho ra cuốn sách cuối cùng mà user này nghe
3. Cho ra bảng thông tin liên quan về các loại sách với nhau
4. Từ cuốn sách cuối cùng mà ng này đọc - recommend cho cuốn sách có lift cao tương tự

### tạo các bảng market basket
Các bước cần làm,
làm trước cho total table sau đó làm riêng cho từng cluster
- tạo dòng đơn hàng và sản phẩm (row là userID, column là playlist)
- biến giá trị thành 0 và 1
- lọc ra những ng nghe từ 2 cuốn trở lên
- ra table frequently (để ra support của mỗi cuốn sách)
- tính ra bảng mỗi liên hệ giữa các sản phẩm với nhau


In [338]:
# Chỉ quan tâm đến có nghe thiệt không chứ không phải bấm lộn, time >= 3 
def encode_time(x):
    if x>=3:
        return x
    else:
        return 0

# Chỉ quan tâm đến có nghe hay không, encode data => có nghe = 1
def encode(x):
    if x>0:
        return 1
    else:
        return 0
    
def mb_frequent(df,list_userid,min_sup = 0.1):
#     dùng df sau khi đã được lọc thành 0,1 và có từ 2 playlist trở lên
    df = df[df.index.isin(list_userid)]
    frequent = apriori(df, min_support = min_sup,
                        use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)
    frequent['length'] = frequent['itemsets'].apply(lambda x:len(x))
    return frequent

In [331]:
# biến thành bảng basket_table trước, sau đó lọc paid user ra
basket = listening.groupby(['userID','playlist_name'])['actual_duation'].sum().unstack().reset_index().fillna(0).set_index('userID')
# quan tâm đến nghe thực >3
basket = basket.applymap(encode_time) 
# biến thành nghe và không nghe 0 và 1
basket = basket.applymap(encode)
# chỉ lấy user nào nghe từ 2 nhóm trở lên
basket = basket[(basket >0).sum(axis=1) >=2]
basket

playlist_name,Bé Gấu 5 Tuổi Đã Dạy Tôi Những Gì,Kinh Vu Lan Bồn,Triết - 101 Đoá Sen,Triết - Trí Tuệ Hoan Hỷ,Triết - Đức Phật Thuyết Giảng Về Tánh Không Và Vô Ngã,Đồng Dao Cho Bé 2,Đồng Dao Cho Bé 3,"""Phúc"" Nuôi Dạy Con",#HOHATUTA Học hành và trưởng thành cùng Hà Anh,"10 Bí Quyết Của Các MC, Diễn Giả",...,Độc Đáo TV - Tin tức hàng ngày,Đội Thiếu Niên Du Kích Thành Huế,Đội Thiếu Niên Du Kích Đình Bảng,Đội Thiếu Niên Tình Báo Bát Sắt,Đột Phá Tư Duy Thương Hiệu,Đừng Bao Giờ Từ Bỏ Ước Mơ,Đừng Mơ Sói Ạ !,Đừng để mất bò,Ưu Đãi Đặc Biệt,Ứng Dụng Thành Công NLP
userID,,,,,,,,,,,,,,,,,,,,,
40839,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40840,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
40842,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40843,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40844,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107068,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
107120,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
107121,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [327]:
# list_userid = list(hv_paid[hv_paid.main_clustering_description == cluster].userID)

In [340]:
def get_list(df,value,col):
    list_userid = list(df[df[col] == value]['userID'])
    return list_userid

In [358]:
clustering.head()

,userID,type_user,main_cluseringID,main_clustering_description
0,40834,free,1,Hibernating
1,40854,free,1,Hibernating
2,40876,free,1,Hibernating
3,40883,free,1,Hibernating
4,40891,free,1,Hibernating


In [360]:
list_hv_paid = ['Loyal Users','Potential Users','New Paid Users']

In [435]:
list_userid = get_list(hv_paid,'New Paid Users','main_clustering_description')
new_frequent = mb_frequent(basket,list_userid,min_sup = 0.01)
new_frequent.head(10)

,support,itemsets,length
0,0.565495,(Muôn Kiếp Nhân Sinh - Tập 1),1
1,0.242812,(Hành Trình Về Phương Đông),1
2,0.198083,"(Hành Trình Về Phương Đông, Muôn Kiếp Nhân Sin...",2
3,0.169329,(Đắc Nhân Tâm),1
4,0.150160,(Thái độ quyết định thành công),1
5,0.146965,(Dấu Chân Trên Cát),1
6,0.127796,(Trời Tính Không Bằng Trời Tính ),1
7,0.124601,"(Dấu Chân Trên Cát, Muôn Kiếp Nhân Sinh - Tập 1)",2
8,0.121406,(Minh Triết Trong Đời Sống),1
9,0.102236,"(Minh Triết Trong Đời Sống, Muôn Kiếp Nhân Sin...",2


In [436]:
# new_frequent = new_frequent[new_frequent.length == 1]
# new_frequent

In [539]:
list_userid2 = get_list(hv_paid,'Potential Users','main_clustering_description')
new_frequent2 = mb_frequent(basket,list_userid2,min_sup = 0.01)
# new_frequent2 = new_frequent2[new_frequent2.length == 1]

In [363]:
list_userid3 = get_list(hv_paid,'Loyal Users','main_clustering_description')
new_frequent3 = mb_frequent(basket,list_userid3,min_sup = 0.01)
new_frequent3 

,support,itemsets,length
0,0.736111,(Muôn Kiếp Nhân Sinh - Tập 1),1
1,0.420635,(Hành Trình Về Phương Đông),1
2,0.380952,"(Hành Trình Về Phương Đông, Muôn Kiếp Nhân Sin...",2
3,0.311508,(Dấu Chân Trên Cát),1
4,0.297619,"(Dấu Chân Trên Cát, Muôn Kiếp Nhân Sinh - Tập 1)",2
...,...,...,...
31685,0.011905,"(Hạnh phúc đến từ sự biến mất, Mật Mã Sự Sống,...",4
31686,0.011905,"(Hạnh phúc đến từ sự biến mất, Sự Thực Về Giác...",4
31687,0.011905,"(Hạnh phúc đến từ sự biến mất, Huyền thuật và ...",4
31688,0.011905,"(Hạnh phúc đến từ sự biến mất, Vị Tu Sĩ Bán Ch...",4


In [432]:
def final_mb(df):
    final_mb = association_rules(df, metric='lift',
                 min_threshold=0.1).sort_values('lift',ascending=False).reset_index(drop=True)
    return final_mb

### New paid user

In [437]:
final_mb1 = final_mb(new_frequent)
final_mb1

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(Nghìn Lẻ Một Đêm - 1001 đêm, Dấu Chân Trên Cá...",(Thần Thoại Hy Lạp),0.015974,0.025559,0.01278,0.800000,31.300000,0.012371,4.872204
1,"(Nghìn Lẻ Một Đêm - 1001 đêm, Dấu Chân Trên Cát)","(Thần Thoại Hy Lạp, Muôn Kiếp Nhân Sinh - Tập 1)",0.015974,0.025559,0.01278,0.800000,31.300000,0.012371,4.872204
2,"(Nghìn Lẻ Một Đêm - 1001 đêm, Dấu Chân Trên Cát)",(Thần Thoại Hy Lạp),0.015974,0.025559,0.01278,0.800000,31.300000,0.012371,4.872204
3,(Thần Thoại Hy Lạp),"(Nghìn Lẻ Một Đêm - 1001 đêm, Dấu Chân Trên Cát)",0.025559,0.015974,0.01278,0.500000,31.300000,0.012371,1.968051
4,"(Thần Thoại Hy Lạp, Muôn Kiếp Nhân Sinh - Tập 1)","(Nghìn Lẻ Một Đêm - 1001 đêm, Dấu Chân Trên Cát)",0.025559,0.015974,0.01278,0.500000,31.300000,0.012371,1.968051
...,...,...,...,...,...,...,...,...,...
1947,(Muôn Kiếp Nhân Sinh - Tập 1),"(Nói Ít Lại, Làm Nhiều Hơn)",0.565495,0.041534,0.01278,0.022599,0.544111,-0.010707,0.980628
1948,(Muôn Kiếp Nhân Sinh - Tập 1),([Sách Tóm Tắt] Ngày Xưa Có Một Con Bò),0.565495,0.047923,0.01278,0.022599,0.471563,-0.014321,0.974090
1949,([Sách Tóm Tắt] Ngày Xưa Có Một Con Bò),(Muôn Kiếp Nhân Sinh - Tập 1),0.047923,0.565495,0.01278,0.266667,0.471563,-0.014321,0.592507
1950,(Thái độ quyết định thành công),"(Hành Trình Về Phương Đông, Muôn Kiếp Nhân Sin...",0.150160,0.198083,0.01278,0.085106,0.429650,-0.016965,0.876514


In [543]:
new = final_mb3
new.antecedents = new.antecedents.apply(lambda x: list(x))
new.consequents = new.consequents.apply(lambda x: list(x))
# new = new[new.consequents.apply(lambda x: list(x))]
new.to_csv('bundle_loyal_users.csv',index=False)

In [542]:
pd.read_csv('bundle_loyal_users.csv')

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"['Nghìn Lẻ Một Đêm - 1001 đêm', 'Dấu Chân Trên...",['Thần Thoại Hy Lạp'],0.015974,0.025559,0.01278,0.800000,31.300000,0.012371,4.872204
1,"['Nghìn Lẻ Một Đêm - 1001 đêm', 'Dấu Chân Trên...","['Thần Thoại Hy Lạp', 'Muôn Kiếp Nhân Sinh - T...",0.015974,0.025559,0.01278,0.800000,31.300000,0.012371,4.872204
2,"['Nghìn Lẻ Một Đêm - 1001 đêm', 'Dấu Chân Trên...",['Thần Thoại Hy Lạp'],0.015974,0.025559,0.01278,0.800000,31.300000,0.012371,4.872204
3,['Thần Thoại Hy Lạp'],"['Nghìn Lẻ Một Đêm - 1001 đêm', 'Dấu Chân Trên...",0.025559,0.015974,0.01278,0.500000,31.300000,0.012371,1.968051
4,"['Thần Thoại Hy Lạp', 'Muôn Kiếp Nhân Sinh - T...","['Nghìn Lẻ Một Đêm - 1001 đêm', 'Dấu Chân Trên...",0.025559,0.015974,0.01278,0.500000,31.300000,0.012371,1.968051
...,...,...,...,...,...,...,...,...,...
1947,['Muôn Kiếp Nhân Sinh - Tập 1'],"['Nói Ít Lại, Làm Nhiều Hơn']",0.565495,0.041534,0.01278,0.022599,0.544111,-0.010707,0.980628
1948,['Muôn Kiếp Nhân Sinh - Tập 1'],['[Sách Tóm Tắt] Ngày Xưa Có Một Con Bò'],0.565495,0.047923,0.01278,0.022599,0.471563,-0.014321,0.974090
1949,['[Sách Tóm Tắt] Ngày Xưa Có Một Con Bò'],['Muôn Kiếp Nhân Sinh - Tập 1'],0.047923,0.565495,0.01278,0.266667,0.471563,-0.014321,0.592507
1950,['Thái độ quyết định thành công'],"['Hành Trình Về Phương Đông', 'Muôn Kiếp Nhân ...",0.150160,0.198083,0.01278,0.085106,0.429650,-0.016965,0.876514


In [ ]:
# dựa trên 1 cuốn sách này, recommend cuốn khác

In [540]:
final_mb2 = final_mb(new_frequent2)
final_mb2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(Sự Tích Sơn Tinh Thủy Tinh, Sự Tích Con Rồng ...","(Sự Tích Thạch Sùng, Thạch Sanh, Chưa Đỗ Ông N...",0.011864,0.010169,0.010169,0.857143,84.285714,0.010049,6.928814
1,"(Sự Tích Trầu Cau, Sự Tích Con Rồng Cháu Tiên,...","(Sự Tích Thạch Sùng, Ăn Khế Trả Vàng, Chưa Đỗ ...",0.011864,0.010169,0.010169,0.857143,84.285714,0.010049,6.928814
2,"(Sự Tích Con Rồng Cháu Tiên, Sự Tích Hồ Gươm)","(Sự Tích Thạch Sùng, Ăn Khế Trả Vàng, Sự Tích ...",0.011864,0.010169,0.010169,0.857143,84.285714,0.010049,6.928814
3,"(Sự Tích Con Rồng Cháu Tiên, Chưa Đỗ Ông Nghè ...","(Sự Tích Thạch Sùng, Ăn Khế Trả Vàng, Sự Tích ...",0.011864,0.010169,0.010169,0.857143,84.285714,0.010049,6.928814
4,"(Sự Tích Trầu Cau, Sự Tích Hồ Gươm)","(Sự Tích Thạch Sùng, Ăn Khế Trả Vàng, Sự Tích ...",0.011864,0.010169,0.010169,0.857143,84.285714,0.010049,6.928814
...,...,...,...,...,...,...,...,...,...
32253,(Muôn Kiếp Nhân Sinh - Tập 1),([Sách Tóm Tắt] Dạy Con Làm Giàu - Cha giàu Ch...,0.647458,0.035593,0.015254,0.023560,0.661930,-0.007791,0.987677
32254,(Trở Về Từ Cõi Sáng),(Đắc Nhân Tâm),0.086441,0.211864,0.011864,0.137255,0.647843,-0.006449,0.913521
32255,(Đắc Nhân Tâm),(Trở Về Từ Cõi Sáng),0.211864,0.086441,0.011864,0.056000,0.647843,-0.006449,0.967754
32256,"(Dấu Chân Trên Cát, Muôn Kiếp Nhân Sinh - Tập 1)",(Thái độ quyết định thành công),0.133898,0.137288,0.010169,0.075949,0.553211,-0.008213,0.933620


In [369]:
final_mb3 = final_mb(new_frequent3)
final_mb3

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(Sự Tích Thạch Sùng, Sự Tích Dưa Hấu)","(Ăn Khế Trả Vàng, Sự Tích Con Rồng Cháu Tiên, ...",0.011905,0.013889,0.011905,1.000000,72.0,0.011739,inf
1,"(Ăn Khế Trả Vàng, Sự Tích Dưa Hấu, Cây Tre Tră...","(Sự Tích Trầu Cau, Sự Tích Con Rồng Cháu Tiên,...",0.011905,0.013889,0.011905,1.000000,72.0,0.011739,inf
2,"(Sự Tích Trầu Cau, Sự Tích Con Rồng Cháu Tiên,...","(Ăn Khế Trả Vàng, Sự Tích Dưa Hấu)",0.013889,0.011905,0.011905,0.857143,72.0,0.011739,6.916667
3,"(Ăn Khế Trả Vàng, Thạch Sanh, Sự Tích Dưa Hấu,...","(Sự Tích Con Rồng Cháu Tiên, Sự Tích Hồ Gươm, ...",0.011905,0.013889,0.011905,1.000000,72.0,0.011739,inf
4,"(Ăn Khế Trả Vàng, Thạch Sanh, Sự Tích Dưa Hấu,...","(Sự Tích Con Rồng Cháu Tiên, Sự Tích Hồ Gươm, ...",0.011905,0.013889,0.011905,1.000000,72.0,0.011739,inf
...,...,...,...,...,...,...,...,...,...
990879,"(Hành Trình Về Phương Đông, Muôn Kiếp Nhân Sin...","(Nói Ít Lại, Làm Nhiều Hơn)",0.380952,0.083333,0.031746,0.083333,1.0,0.000000,1.000000
990880,(The Four - Tứ Đại Quyền Lực),"(Hành Trình Về Phương Đông, Muôn Kiếp Nhân Sin...",0.041667,0.380952,0.015873,0.380952,1.0,0.000000,1.000000
990881,(Tuổi 40 Trải Nghiệm),"(Hành Trình Về Phương Đông, Huyền thuật và các...",0.111111,0.125000,0.013889,0.125000,1.0,0.000000,1.000000
990882,"(Hành Trình Về Phương Đông, Muôn Kiếp Nhân Sin...",(The Four - Tứ Đại Quyền Lực),0.380952,0.041667,0.015873,0.041667,1.0,0.000000,1.000000


In [371]:
clustering[clustering.type_user == 'paid']

,userID,type_user,main_cluseringID,main_clustering_description
24820,40843,paid,5,New Paid Users
24821,41955,paid,5,New Paid Users
24822,44112,paid,5,New Paid Users
24823,45239,paid,5,New Paid Users
24824,45647,paid,5,New Paid Users
...,...,...,...,...
27453,76980,paid,3,Lost Users
27454,78868,paid,3,Lost Users
27455,81001,paid,3,Lost Users
27456,83850,paid,3,Lost Users


In [375]:
listening[listening.userID == 44112].sort_values('listening_date',ascending=False)

,playlistID,playlist_name,cat,sub_cat,palylist_type,playlist_duration,actual_duation,userID,listening_time,listening_date,listening_datetime,Unnamed: 11,by_week
63811,5140,Thiên Nhiên Liều Thuốc Nhiệm Màu,Sách nói,Sức khoẻ,free,177.0,7.3,44112,12:09:00,31-12-2020,2020-12-31 12:09:00,NaN,2020-12-28 12:09:00
230041,4407,Nghìn Lẻ Một Đêm - 1001 đêm,Thiếu nhi,Thiếu nhi,free,125.6,1.8,44112,06:51:00,31-10-2020,2020-10-31 06:51:00,NaN,2020-10-26 06:51:00
316420,4407,Nghìn Lẻ Một Đêm - 1001 đêm,Thiếu nhi,Thiếu nhi,free,125.6,2.8,44112,21:22:00,31-10-2020,2020-10-31 21:22:00,NaN,2020-10-26 21:22:00
545052,4407,Nghìn Lẻ Một Đêm - 1001 đêm,Thiếu nhi,Thiếu nhi,free,125.6,4.8,44112,21:22:00,30-09-2020,2020-09-30 21:22:00,NaN,2020-09-28 21:22:00
282191,4395,Chàng Ngốc Ivan,Thiếu nhi,Thiếu nhi,free,23.4,8.2,44112,18:29:00,29-12-2020,2020-12-29 18:29:00,NaN,2020-12-28 18:29:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
363414,4280,Truyện Ngắn Thạch Lam 2 - Bóng Người Xưa,Truyện nói,Kinh điển Việt Nam,free,193.0,0.6,44112,22:16:00,03-09-2020,2020-09-03 22:16:00,NaN,2020-08-31 22:16:00
407923,4407,Nghìn Lẻ Một Đêm - 1001 đêm,Thiếu nhi,Thiếu nhi,free,125.6,0.0,44112,17:49:00,02-10-2020,2020-10-02 17:49:00,NaN,2020-09-28 17:49:00
342469,4365,Đồng Dao Vui Chơi 1,Thiếu nhi,Thiếu nhi,free,4.7,0.5,44112,15:14:00,01-12-2020,2020-12-01 15:14:00,NaN,2020-11-30 15:14:00
382359,4326,Sống Xanh - Bài Ca Của Cây,Thiếu nhi,Thiếu nhi,free,13.4,1.2,44112,17:11:00,01-12-2020,2020-12-01 17:11:00,NaN,2020-11-30 17:11:00


In [616]:
def check_set(df,col,playlist,type_str = 1 ):
    index = -1
    list_index_group = list()
    if type_str == 1:
#         print('ok')
        for i in range(len(df[col])):
            set_item = list(df[col][i])
            for k in range(len(set_item)):
                if set_item[k] == playlist:                                
                    if len(set_item) == 1:
                        index = i
                    else: 
                        list_index_group.append(i)
    #                 print (k)
    #                 print(i)
    #                 print(set_item[k])
    else:
        for i in range(len(df[col])):
            set_item = list(df[col][i])
            if set_item == playlist:
                index = i
        
    return (list_index_group,index)
#     return
#     print(i)

In [438]:
final_mb1.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(Nghìn Lẻ Một Đêm - 1001 đêm, Dấu Chân Trên Cá...",(Thần Thoại Hy Lạp),0.015974,0.025559,0.01278,0.8,31.3,0.012371,4.872204
1,"(Nghìn Lẻ Một Đêm - 1001 đêm, Dấu Chân Trên Cát)","(Thần Thoại Hy Lạp, Muôn Kiếp Nhân Sinh - Tập 1)",0.015974,0.025559,0.01278,0.8,31.3,0.012371,4.872204
2,"(Nghìn Lẻ Một Đêm - 1001 đêm, Dấu Chân Trên Cát)",(Thần Thoại Hy Lạp),0.015974,0.025559,0.01278,0.8,31.3,0.012371,4.872204
3,(Thần Thoại Hy Lạp),"(Nghìn Lẻ Một Đêm - 1001 đêm, Dấu Chân Trên Cát)",0.025559,0.015974,0.01278,0.5,31.3,0.012371,1.968051
4,"(Thần Thoại Hy Lạp, Muôn Kiếp Nhân Sinh - Tập 1)","(Nghìn Lẻ Một Đêm - 1001 đêm, Dấu Chân Trên Cát)",0.025559,0.015974,0.01278,0.5,31.3,0.012371,1.968051


In [419]:
# giờ thử cho 1 cái tựa đề sách + user id => recommend
# new paid user
# userID = 44112 => list playlist
# playlist = 'Thiên Nhiên Liều Thuốc Nhiệm Màu'
playlist = 'Nghìn Lẻ Một Đêm - 1001 đêm'
# new_frequent[new_frequent['itemsets'].isin(('(Muôn Kiếp Nhân Sinh - Tập 1)','a'))]

### to recommend sach
- Lấy list playlist đã nghe
- Check trong bảng sáng phổ biến có không
- Nếu có (len = 1) => suggest những cái bundel đi kèm (lấy cái cao nhất - lift)
- Nếu có (len >1) => check tiếp những thành phần khác trong combo đó
    Nếu full => suggest bundel đi kèm (lấy cái cao nhất - lift)
    Nếu half => suggest những sách còn lại cùng bundel (hiện support của nhóm)

In [675]:
# dùng để tạo ra 1 bảng recommend sách dựa trên 1 đầu sách vào
def creat_recom_full(df,i,index,loai_ac='A'):
    table = []
    da_doc =i 
#         print(type(final_mb1.iloc[check[1]]))
    sr = df.iloc[index]
#         print(sr)           
    de_xuat=list(sr.consequents)
    loai=loai_ac
    confidence=sr.confidence
    lift=sr.lift    
    support=sr.support
    return [da_doc,de_xuat,loai,confidence,lift,support]


In [548]:
# check list đi kèm đã trong list đọc chưa
def check_list_da_doc(list_check,list_goc):
    for i in list_check:
        if i in list_goc:
            result = 1
        else:
            result = 0
            break
    return result

In [695]:
# final
# df là bảng frequent
def get_recommendation (userID_input,clus_input):
    if clus_input == 'New Paid Users':
        frequent = new_frequent
        basket = final_mb1
    elif clus_input == 'Potential Users':
        frequent = new_frequent2
        basket = final_mb2
    else:
        frequent = new_frequent3
        basket = final_mb3
    list_playlist = list(set(listening[listening.userID == userID_input].playlist_name))
#     lọc ra playlist đơn và nhóm
    list_one = list()
    list_bundle = list()

    for i in list_playlist:
        check = check_set(frequent,'itemsets',i)
        if check[1] != -1:
            list_one.append(i)
        if check[0] != []:
            list_bundle.append(i)
    #         print(check)

# với những playlist xuất hiện đơn, recommend những cuốn sách dựa trên lift
    title = ['da_doc','de_xuat','loai','confidence','lift','support']
    total_don = []

    for i in list_one:
        check = check_set(basket,'antecedents',i)
    #     print(check)
        if check[1] != -1:
            table_1= creat_recom_full(basket,i,check[1],loai_ac='A')
            if table_1[4] >1 and check_list_da_doc(table_1[1],list_playlist) == 0:
                total_don.append(table_1)
    table_don = DataFrame(total_don,columns=title)
    table_don=table_don.sort_values('lift')
# đối vs nhóm sách xuất hiện theo nhóm trong bảng frequent    
    total_nhom = list()
    for i in list_bundle:
#         print(i)
        check = check_set(frequent,'itemsets',i,type_str=1)
        for k in check[0]:
            di_kem = list(frequent.iloc[k]['itemsets'])
    #         print('Bundle ',di_kem)
            if check_list_da_doc(di_kem,list_playlist) == 0:
    #             print('Bộ sách thường được đọc cùng nhau ',di_kem)
    #             print('Bộ sách thường được đọc cùng nhau ',new_frequent.iloc[k]['support'])
    #             print('Look here')
    #             print(da_doc)
    #             print(di_kem)
                da_doc = i
                di_kem.remove(i)
                de_xuat = di_kem
                support = frequent.iloc[k]['support']
                total_nhom.append([da_doc,de_xuat,0,0,0,support])
    #             print('de_xuat',de_xuat)
            else:
                check = check_set(basket,'antecedents',test,type_str=0)
                if check[1] != -1:
    #                 print (check[1])
                    table_1= creat_recom_full(basket,i,check[1],loai_ac='A')
                    total_nhom.append(table_1)
    table_nhom = DataFrame(total_nhom,columns=title)    
    final = DataFrame(total_don+total_nhom,columns=title)
    return final

In [718]:
final_table = get_recommendation (45647,'New Paid Users').head(10)
for i in range(0,3):
    
    r1 = final_table.iloc[i].to_list()
#     print(r1)
    print(f'User đã đọc {r1[0]}')
    print(f'Khuyến nghị user nên đọc {r1[1]}')
    if r1[4] > 1:
        print('Tỉ lệ suất hiện cùng nhau lift cao',r1[4])

    if r1[3] > 0.2:
        print(f'{r1[3]*100} % người cùng nhóm với user đã đọc {r1[0]} và cũng đã đọc {r1[1]}')
    print(f'Tỉ lệ số người cùng đọc combo {r1[5]*100}')
    print('----------------------------------------------------')

User đã đọc Hành Trình Về Phương Đông
Khuyến nghị user nên đọc ['Thần Thoại Hy Lạp']
Tỉ lệ suất hiện cùng nhau lift cao 31.300000000000004
80.0 % người cùng nhóm với user đã đọc Hành Trình Về Phương Đông và cũng đã đọc ['Thần Thoại Hy Lạp']
Tỉ lệ số người cùng đọc combo 1.2779552715654952
----------------------------------------------------
User đã đọc Hành Trình Về Phương Đông
Khuyến nghị user nên đọc ['Dấu Chân Trên Cát']
Tỉ lệ số người cùng đọc combo 7.348242811501597
----------------------------------------------------
User đã đọc Hành Trình Về Phương Đông
Khuyến nghị user nên đọc ['Dấu Chân Trên Cát', 'Muôn Kiếp Nhân Sinh - Tập 1']
Tỉ lệ số người cùng đọc combo 7.0287539936102235
----------------------------------------------------
